In [ ]:
import torch
import torchvision

In [ ]:
torch.__version__, torchvision.__version__

In [ ]:
!pip install -q git+https://github.com/google-deepmind/tapnet.git

In [ ]:
!pip install -q git+https://github.com/google-deepmind/recurrentgemma.git@main

In [ ]:
!pip install "numpy<2.1.0"

In [ ]:
import tqdm

### TAPNext

In [ ]:
import time
import numpy as np
from tapnet.tapnext.tapnext_torch import TAPNext
import torch.nn.functional as F

### Create the model and load checkpoint

In [ ]:
model = TAPNext(image_size=(256, 256))
model.cuda()

### Run inference

In [ ]:
model.eval()
for p in model.parameters():
  p.requires_grad = False

In [ ]:
NUM_QUERIES = 1024

In [ ]:
video = torch.zeros((1, 1024, 256, 256, 3), dtype=torch.float32).cuda()
query_points = torch.zeros((1, NUM_QUERIES, 3), dtype=torch.float32).cuda()

In [ ]:
DTYPE = torch.float16  # use fp16 or bf16

In [ ]:
fwd = torch.compile(model.forward)
with torch.no_grad():
  with torch.amp.autocast('cuda', dtype=DTYPE, enabled=True):
    _, _, _, tracking_state = fwd(video=video[:, :1], query_points=query_points)
    c = 0
    for k in tqdm.tqdm(range(1, video.shape[1])):
      if k == 512:
        # we let the model to run for several GPU burn-in steps
        tt = time.time()
        c = 0
      _, _, _, tracking_state = fwd(
          video=video[:, k : k + 1], state=tracking_state
      )
      c += 1
    d = time.time() - tt
    print('FPS:', c / d, 'latency', 1000 * d / c, 'ms')